<a href="https://colab.research.google.com/github/apgis/next_day_temp_predictor/blob/main/Temp_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense

In [ ]:
temperatures = np.array([30.1, 30.5, 31.0, 31.4, 32.0, 31.6, 33.2, 34.0, 33.8, 35.2,
                         36.0, 36.5, 37.1, 38.0, 38.2, 37.9, 37.0, 36.4, 35.0, 34.5])

In [12]:
seq_len = 5
X = []
y = []

for i in range(len(temperatures) - seq_len):
    X.append(temperatures[i:i+seq_len])
    y.append(temperatures[i+seq_len])

X = np.array(X)
y = np.array(y)


# Reshape X to (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X)

[[[30.1]
  [30.5]
  [31. ]
  [31.4]
  [32. ]]

 [[30.5]
  [31. ]
  [31.4]
  [32. ]
  [31.6]]

 [[31. ]
  [31.4]
  [32. ]
  [31.6]
  [33.2]]

 [[31.4]
  [32. ]
  [31.6]
  [33.2]
  [34. ]]

 [[32. ]
  [31.6]
  [33.2]
  [34. ]
  [33.8]]

 [[31.6]
  [33.2]
  [34. ]
  [33.8]
  [35.2]]

 [[33.2]
  [34. ]
  [33.8]
  [35.2]
  [36. ]]

 [[34. ]
  [33.8]
  [35.2]
  [36. ]
  [36.5]]

 [[33.8]
  [35.2]
  [36. ]
  [36.5]
  [37.1]]

 [[35.2]
  [36. ]
  [36.5]
  [37.1]
  [38. ]]

 [[36. ]
  [36.5]
  [37.1]
  [38. ]
  [38.2]]

 [[36.5]
  [37.1]
  [38. ]
  [38.2]
  [37.9]]

 [[37.1]
  [38. ]
  [38.2]
  [37.9]
  [37. ]]

 [[38. ]
  [38.2]
  [37.9]
  [37. ]
  [36.4]]

 [[38.2]
  [37.9]
  [37. ]
  [36.4]
  [35. ]]]


In [18]:
model=Sequential([
    Bidirectional(LSTM(64,input_shape=(seq_len,1))),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
model.compile(optimizer='adam',loss='mse')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X,y,epochs=100,verbose=0)


In [23]:
# Last 5 known temps to predict the next
last_5_days = np.array([37.0, 36.4, 35.0, 34.5, 34.0]).reshape(1, seq_len, 1)
predicted_temp = model.predict(last_5_days)

print(f"Predicted temperature for tomorrow: {predicted_temp[0][0]:.2f}°C")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
Predicted temperature for tomorrow: 35.31°C


In [25]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00


In [27]:
import gradio as gr

In [28]:
def predict_temperature(t1, t2, t3, t4, t5):
    input_seq = np.array([t1, t2, t3, t4, t5]).reshape(1, seq_len, 1)
    pred = model.predict(input_seq)[0][0]
    return f"Predicted temperature for next day: {pred:.2f}°C"

# -----------------------------
# Gradio UI
# -----------------------------
inputs = [gr.Number(label=f"Day {i+1} Temp (°C)") for i in range(5)]
interface = gr.Interface(fn=predict_temperature, inputs=inputs, outputs="text", title="Next-Day Temperature Predictor")

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://25e85d8b979d287261.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


SyntaxError: invalid syntax (<ipython-input-29-5bb206e09d22>, line 1)